# DART 오픈 API 01 - 개요와 활용

- DART 오픈AP 사용법
- json_normalize() 활용 
- 기업개황 API
- 정보검색 API
- 개별 보고서와 하위 보고서 URL 얻기
- 첨부파일과 재무제표 엑셀 파일 가져오기

### 오픈 API

- 공시 보고서 목록를 실시간 제공
- https://dart.fss.or.kr/dsap001/intro.do 오픈API 소개
- https://dart.fss.or.kr/dsap001/guide.do API 개발 가이드

### API 인증키 발급

- 계정을 만들고, API 인증키 발급 받는다

- 계정 신청: https://dart.fss.or.kr/dsag002/insertForm.do

### 개인용, 기업용
- 개인용: 1만건/일
- 기업용: 조회 건수 제한이 없는 대신 조회 서버 IP 제한

### 인증키
- 알파벳과 숫자로 이루어진 40 바이트 문자열
- auth 파라미터로 제공

http://dart.fss.or.kr/api/search.xml?auth=e81e78aa519d1a1c4ec7867ef12a937ab6cbd4c7

### 오픈 API가 제공하는 주요정보

 - 주요하게 다음 2가지 API를 제공한다
    1. 기업 개황 API : http://dart.fss.or.kr/api/company.json?auth=xxx&crp_cd=xxx
    2. 정보 검색 API : http://dart.fss.or.kr/api/search.json?auth=xxx



## API 개발 가이드 사이트에서 테스트

- https://dart.fss.or.kr/dsap001/guide.do API 개발 가이드

![](http://i.imgur.com/9NpoJgx.png)

In [2]:
## 현재 발급받은 API인증키(2018-06-03)
auth = '9daed961201d085d01dfb40c8a1acb5c3fd41b85'

# 헤더: 실제 인증키를 사용시에는 사용하지 않아도 된다
headers={'Referer':'https://dart.fss.or.kr/dsap001/guide.do'}

In [9]:
import requests

url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}'
url = url_tmpl.format(auth=auth)

r = requests.get(url, headers=headers)
r.text

'{"err_code":"000","err_msg":"정상","page_no":1,"page_set":10,"total_count":0,"total_page":0,"list":[]}'

In [25]:
from pandas.io.json import json_normalize
import json
import pandas as pd


json_str = '''
[
    {"A":1, "B":2, "C":3},
    {"A":4, "B":5, "C":6},
    {"A":7, "B":8, "C":9}
]
'''

jo = json.loads(json_str)
result = json_normalize(jo)
result

,A,B,C
0,1,2,3
1,4,5,6
2,7,8,9


In [14]:
url = 'http://dart.fss.or.kr/api/company.json?auth={auth}&crp_cd={code}'
url = url.format(auth=auth, code ='005930')
r = requests.get(url)
jo = json.loads(r.text)
jo

{'acc_mt': '12',
 'adr': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'bsn_no': '1248100998',
 'ceo_nm': '김기남, 김현석, 고동진',
 'crp_cls': 'Y',
 'crp_nm': '삼성전자(주)',
 'crp_nm_e': 'SAMSUNG ELECTRONICS CO,.LTD',
 'crp_nm_i': '삼성전자',
 'crp_no': '1301110006246',
 'err_code': '000',
 'err_msg': '정상',
 'est_dt': '19690113',
 'fax_no': '031-200-7538',
 'hm_url': 'www.sec.co.kr',
 'ind_cd': '264',
 'ir_url': '',
 'phn_no': '031-200-1114',
 'stock_cd': '005930'}

### 위 결과물의 내용
- err_code: 에러코드(오류 메시지 참조)
- err_msg: 에러메시지(오류 메시지 참조)
- crp_nm: 정식명칭
- crp_nm_e: 영문명칭
- crp_nm_i: 종목명(상장사) 또는 약식명칭(기타법인)
- stock_cd: 상장회사인 경우 주식의 종목코드
- ceo_nm: 대표자명
- crp_cls: 법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)
- crp_no: 법인등록번호
- bsn_no: 사업자등록번호
- adr: 주소
- hm_url: 홈페이지
- ir_url: IR홈페이지
- phn_no: 전화번호
- fax_no: 팩스번호
- ind_cd: 업종코드
- est_dt: 설립일(YYYYMMDD)
- acc_mt: 결산월(MM)


In [16]:
jo['bsn_no'] #사업자 등록번호

'1248100998'

### 정보 검색 API

- http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003 
    - (회사의 전체 사업+반기+분기보고서)
- crp_cd: 공시대상회사의 종목코드(상장사:숫자 6자리) 또는 고유번호(기타법인:숫자 8자리)
- end_dt: 검색종료 접수일자(YYYYMMDD) : 없으면 당일
- start_dt: 검색시작 접수일자(YYYYMMDD) : 없으면 end_dt crp_cd가 없는 경우 검색기간은 3개월로 제한
- fin_rpt: 최종보고서만 검색여부(Y or N) 기본값 : N (정정이 있는 경우 최종정정만 검색)
- dsp_tp: 정기공시(A), 주요사항보고(B), 발행공시(C), 지분공시(D), 기타공시(E), 외부감사관련(F), 펀드공시(G), 자산유동화(H), 거래소공시(I), 공정위공시(J)
- bsn_tp: 정기공시(5), 주요사항보고(3), 발행공시(11), 지분공시(4), 기타공시(9), 외부감사관련(3), 펀드공시(3), 자산유동화(6), 거래소공시(6), 공정위공시(5) 
- sort: 접수일자(date), 회사명(crp), 보고서명(rpt) 기본값 : date
- series: 오름차순(asc), 내림차순(desc) 기본값 : desc
- page_no: 페이지 번호(1~n) 기본값 : 1
- page_set: 페이지당 건수(1~100) 기본값 : 10, 최대값 : 100
- callback: 콜백함수명(JSONP용)


In [22]:
url = 'http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003' 
url = url.format(auth=auth, code ='005930')
r = requests.get(url)
jo = json.loads(r.text)
jo

{'err_code': '000',
 'err_msg': '정상',
 'list': [{'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20180515',
   'rcp_no': '20180515001699',
   'rmk': '',
   'rpt_nm': '분기보고서 (2018.03)'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20180402',
   'rcp_no': '20180402005019',
   'rmk': '연',
   'rpt_nm': '사업보고서 (2017.12)'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20171114',
   'rcp_no': '20171114001694',
   'rmk': '',
   'rpt_nm': '분기보고서 (2017.09)'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20170814',
   'rcp_no': '20170814001832',
   'rmk': '',
   'rpt_nm': '반기보고서 (2017.06)'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20170515',
   'rcp_no': '20170515003806',
   'rmk': '',
   'rpt_nm': '분기보고서 (2017.03)'},
  {'crp_cd': '0059

In [30]:
# 당일접수 10건보기(주말 공휴일에는 접수가 없을 수 있음...)

url = 'http://dart.fss.or.kr/api/search.json?auth={auth}&page_set=10'.format(auth=auth)
r = requests.get(url, headers=headers)
jo = json.loads(r.text)
result = json_normalize(jo,'list')
result.head()
jo


{'err_code': '000',
 'err_msg': '정상',
 'list': [],
 'page_no': 1,
 'page_set': 10,
 'total_count': 0,
 'total_page': 0}